In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [25]:
load_dotenv()

True

In [26]:
template = """
You are an expert SQL query writer. 
Given the database schema and the user's question, write the most accurate SQL query to answer the question. 
Use only the tables and columns mentioned in the schema.

Schema:
{schema}

Question:
{question}

SQL Query:
"""
prompt = ChatPromptTemplate.from_template(template)

In [27]:
# Fill in the schema and question placeholders in the prompt
formatted_prompt = prompt.format(
    schema="myschema",                     # Database schema info
    question="How many users are there?"   # User's natural language question
)

# Print the final prompt (ready for the LLM)
print(formatted_prompt)


"Human: \nYou are an expert SQL query writer. \nGiven the database schema and the user's question, write the most accurate SQL query to answer the question. \nUse only the tables and columns mentioned in the schema.\n\nSchema:\nmyschema\n\nQuestion:\nHow many users are there?\n\nSQL Query:\n"

In [28]:
# Define the database connection URI
# Format: "mysql+mysqlconnector://<username>:<password>@<host>:<port>/<database_name>"
db_uri = "mysql+mysqlconnector://root:@localhost:3306/chinook"

# Create a LangChain SQLDatabase object using the connection URI
# This lets LangChain interact directly with your MySQL database
db = SQLDatabase.from_uri(db_uri)

In [29]:
# Define a function to get the database schema (tables and columns)
def get_schema(_):
    # Returns information about all tables in the connected database
    return db.get_table_info()

In [30]:
# Create an instance of the OpenAI chat model (LLM)
llm = ChatOpenAI()

In [31]:
# Create a LangChain pipeline (chain) to generate SQL queries step by step
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)  # Get the database schema
    | prompt                                       # Insert schema & question into the prompt
    | llm.bind(stop="\nSQL Result:")               # Use the LLM to generate the SQL query
    | StrOutputParser()                            # Extract the text output from the LLM
)

In [32]:
# Define a template for generating natural language answers from SQL results
template = """
You are a data analyst. Using the table schema, SQL query, and its result, generate a clear answer to the user's question.

Schema:
{schema}

Question:
{question}

SQL Query:
{query}

SQL Response:
{response}
"""

# Create a ChatPromptTemplate object from the template
prompt = ChatPromptTemplate.from_template(template)

In [35]:
# Define a function to run an SQL query on the connected database
def run_query(query):
    # Executes the given SQL query and returns the result
    return db.run(query)

In [36]:
# Create the full LangChain pipeline to go from question → SQL → result → answer
full_chain = (
    RunnablePassthrough.assign(query=sql_chain)         # Generate the SQL query from the question
    .assign(
        schema=get_schema,                              # Get the database schema
        response=lambda vars: run_query(vars["query"])  # Run the generated SQL query and get results
    )
    | prompt                                             # Fill the final prompt with schema, query, and response
    | llm                                                # Use the LLM to generate a answer
    | StrOutputParser()                                  # Extract plain text output from the model
)


In [37]:
# Send a question to the full chain to generate and run an SQL query
result = full_chain.invoke({
    "question": "Which three countries have the highest total amounts?"
})

# Print the final result
print(result)


In [38]:
print(result)

The three countries with the highest total amounts from invoices are the USA with a total amount of $523.06, followed by Canada with a total amount of $303.96, and France with a total amount of $195.10.
